In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "01_starter_code"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 8
    model_name = 'efficientnet-b0'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  01_starter_code  Created 


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [7]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [8]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.33, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [9]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b0


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True, mode="min", factor=0.3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

# end_lr = 0.1
# start_lr = 1e-7
# lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * len(train_dataloader)))
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# scheduler_1 = LinearCyclicalScheduler(optimizer, "lr", start_value=0.01, end_value=0.005, cycle_size=60)
# scheduler_2 = CosineAnnealingScheduler(optimizer, "lr", start_value=0.05, end_value=0.001, cycle_size=60)
# combined_scheduler = ConcatScheduler(schedulers=[scheduler_1, scheduler_2], durations=[30, ])

In [11]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [12]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 11s
Training Results - Average Loss: 1.0541 | auc/_mean: 0.6832 | auc/healthy: 0.6867 | auc/multiple_diseases: 0.4969, auc/rust: 0.7672, auc/scab: 0.7822
Evaluating Results - Average Loss: 0.6722 | auc/_mean: 0.8379 | auc/healthy: 0.8994 | auc/multiple_diseases: 0.5689, auc/rust: 0.9465, auc/scab: 0.9369




Epoch: 02 | Time: 0m 11s
Training Results - Average Loss: 0.9149 | auc/_mean: 0.7650 | auc/healthy: 0.7833 | auc/multiple_diseases: 0.6067, auc/rust: 0.8386, auc/scab: 0.8312
Evaluating Results - Average Loss: 0.6147 | auc/_mean: 0.8534 | auc/healthy: 0.9178 | auc/multiple_diseases: 0.5988, auc/rust: 0.9540, auc/scab: 0.9429




Epoch: 03 | Time: 0m 11s
Training Results - Average Loss: 0.8780 | auc/_mean: 0.7770 | auc/healthy: 0.8023 | auc/multiple_diseases: 0.6045, auc/rust: 0.8477, auc/scab: 0.8534
Evaluating Results - Average Loss: 0.5491 | auc/_mean: 0.8635 | auc/healthy: 0.9302 | auc/multiple_diseases: 0.6151, auc/rust: 0.9604, auc/scab: 0.9483




Epoch: 04 | Time: 0m 11s
Training Results - Average Loss: 0.8489 | auc/_mean: 0.7893 | auc/healthy: 0.8238 | auc/multiple_diseases: 0.6261, auc/rust: 0.8588, auc/scab: 0.8486
Evaluating Results - Average Loss: 0.5594 | auc/_mean: 0.8583 | auc/healthy: 0.9253 | auc/multiple_diseases: 0.6045, auc/rust: 0.9592, auc/scab: 0.9441




Epoch: 05 | Time: 0m 11s
Training Results - Average Loss: 0.8193 | auc/_mean: 0.8082 | auc/healthy: 0.8340 | auc/multiple_diseases: 0.6704, auc/rust: 0.8692, auc/scab: 0.8591
Evaluating Results - Average Loss: 0.5772 | auc/_mean: 0.8623 | auc/healthy: 0.9266 | auc/multiple_diseases: 0.6210, auc/rust: 0.9503, auc/scab: 0.9515

Epoch     5: reducing learning rate of group 0 to 9.0000e-04.


...


Epoch: 24 | Time: 0m 11s
Training Results - Average Loss: 0.7528 | auc/_mean: 0.8389 | auc/healthy: 0.8773 | auc/multiple_diseases: 0.7475, auc/rust: 0.8680, auc/scab: 0.8629
Evaluating Results - Average Loss: 0.5165 | auc/_mean: 0.8706 | auc/healthy: 0.9375 | auc/multiple_diseases: 0.6363, auc/rust: 0.9579, auc/scab: 0.9508




Epoch: 25 | Time: 0m 11s
Training Results - Average Loss: 0.7697 | auc/_mean: 0.8268 | auc/healthy: 0.8661 | auc/multiple_diseases: 0.7127, auc/rust: 0.8675, auc/scab: 0.8611
Evaluating Results - Average Loss: 0.5174 | auc/_mean: 0.8717 | auc/healthy: 0.9362 | auc/multiple_diseases: 0.6430, auc/rust: 0.9576, auc/scab: 0.9499

Epoch    25: reducing learning rate of group 0 to 2.1870e-06.



Epoch: 26 | Time: 0m 11s
Training Results - Average Loss: 0.7138 | auc/_mean: 0.8384 | auc/healthy: 0.8804 | auc/multiple_diseases: 0.7032, auc/rust: 0.8844, auc/scab: 0.8855
Evaluating Results - Average Loss: 0.5200 | auc/_mean: 0.8713 | auc/healthy: 0.9361 | auc/multiple_diseases: 0.6422, auc/rust: 0.9569, auc/scab: 0.9501




Epoch: 27 | Time: 0m 11s
Training Results - Average Loss: 0.7480 | auc/_mean: 0.8320 | auc/healthy: 0.8602 | auc/multiple_diseases: 0.7156, auc/rust: 0.8741, auc/scab: 0.8778
Evaluating Results - Average Loss: 0.5215 | auc/_mean: 0.8708 | auc/healthy: 0.9356 | auc/multiple_diseases: 0.6430, auc/rust: 0.9563, auc/scab: 0.9482




Epoch: 28 | Time: 0m 11s
Training Results - Average Loss: 0.7472 | auc/_mean: 0.8300 | auc/healthy: 0.8553 | auc/multiple_diseases: 0.7083, auc/rust: 0.8742, auc/scab: 0.8823
Evaluating Results - Average Loss: 0.5185 | auc/_mean: 0.8720 | auc/healthy: 0.9361 | auc/multiple_diseases: 0.6462, auc/rust: 0.9569, auc/scab: 0.9488

EarlyStopping


In [13]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=17_loss=0.5161190929969675.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config.patience, verbose=True, mode="min", factor=0.3)

Loaded pretrained weights for efficientnet-b0


In [14]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 20s
Training Results - Average Loss: 1.1178 | auc/_mean: 0.7296 | auc/healthy: 0.7870 | auc/multiple_diseases: 0.4864, auc/rust: 0.8249, auc/scab: 0.8200
Evaluating Results - Average Loss: 0.9375 | auc/_mean: 0.7582 | auc/healthy: 0.9206 | auc/multiple_diseases: 0.5427, auc/rust: 0.8321, auc/scab: 0.7375




Epoch: 02 | Time: 0m 20s
Training Results - Average Loss: 0.6236 | auc/_mean: 0.8282 | auc/healthy: 0.9341 | auc/multiple_diseases: 0.5124, auc/rust: 0.9531, auc/scab: 0.9133
Evaluating Results - Average Loss: 0.6220 | auc/_mean: 0.9076 | auc/healthy: 0.9817 | auc/multiple_diseases: 0.6713, auc/rust: 0.9935, auc/scab: 0.9840




Epoch: 03 | Time: 0m 20s
Training Results - Average Loss: 0.6093 | auc/_mean: 0.8557 | auc/healthy: 0.9409 | auc/multiple_diseases: 0.6203, auc/rust: 0.9341, auc/scab: 0.9273
Evaluating Results - Average Loss: 0.4952 | auc/_mean: 0.8957 | auc/healthy: 0.9800 | auc/multiple_diseases: 0.6297, auc/rust: 0.9928, auc/scab: 0.9803




Epoch: 04 | Time: 0m 20s
Training Results - Average Loss: 0.5456 | auc/_mean: 0.8580 | auc/healthy: 0.9596 | auc/multiple_diseases: 0.5737, auc/rust: 0.9522, auc/scab: 0.9464
Evaluating Results - Average Loss: 0.3927 | auc/_mean: 0.8538 | auc/healthy: 0.9868 | auc/multiple_diseases: 0.4447, auc/rust: 0.9954, auc/scab: 0.9882




Epoch: 05 | Time: 0m 20s
Training Results - Average Loss: 0.4750 | auc/_mean: 0.8659 | auc/healthy: 0.9655 | auc/multiple_diseases: 0.5825, auc/rust: 0.9713, auc/scab: 0.9445
Evaluating Results - Average Loss: 0.5728 | auc/_mean: 0.8768 | auc/healthy: 0.9837 | auc/multiple_diseases: 0.5457, auc/rust: 0.9943, auc/scab: 0.9833



...


Epoch: 36 | Time: 0m 20s
Training Results - Average Loss: 0.1418 | auc/_mean: 0.9239 | auc/healthy: 0.9985 | auc/multiple_diseases: 0.7061, auc/rust: 0.9962, auc/scab: 0.9948
Evaluating Results - Average Loss: 0.1717 | auc/_mean: 0.9147 | auc/healthy: 0.9975 | auc/multiple_diseases: 0.6671, auc/rust: 0.9991, auc/scab: 0.9952




Epoch: 37 | Time: 0m 20s
Training Results - Average Loss: 0.1520 | auc/_mean: 0.9261 | auc/healthy: 0.9970 | auc/multiple_diseases: 0.7165, auc/rust: 0.9963, auc/scab: 0.9947
Evaluating Results - Average Loss: 0.1735 | auc/_mean: 0.9045 | auc/healthy: 0.9977 | auc/multiple_diseases: 0.6290, auc/rust: 0.9988, auc/scab: 0.9927




Epoch: 38 | Time: 0m 20s
Training Results - Average Loss: 0.1430 | auc/_mean: 0.9334 | auc/healthy: 0.9984 | auc/multiple_diseases: 0.7449, auc/rust: 0.9952, auc/scab: 0.9952
Evaluating Results - Average Loss: 0.1707 | auc/_mean: 0.8860 | auc/healthy: 0.9973 | auc/multiple_diseases: 0.5522, auc/rust: 0.9985, auc/scab: 0.9958




Epoch: 39 | Time: 0m 20s
Training Results - Average Loss: 0.1719 | auc/_mean: 0.9237 | auc/healthy: 0.9969 | auc/multiple_diseases: 0.7118, auc/rust: 0.9939, auc/scab: 0.9921
Evaluating Results - Average Loss: 0.2142 | auc/_mean: 0.9221 | auc/healthy: 0.9955 | auc/multiple_diseases: 0.6985, auc/rust: 0.9990, auc/scab: 0.9952




Epoch: 40 | Time: 0m 20s
Training Results - Average Loss: 0.1643 | auc/_mean: 0.9263 | auc/healthy: 0.9966 | auc/multiple_diseases: 0.7168, auc/rust: 0.9955, auc/scab: 0.9963
Evaluating Results - Average Loss: 0.1998 | auc/_mean: 0.9031 | auc/healthy: 0.9969 | auc/multiple_diseases: 0.6230, auc/rust: 0.9986, auc/scab: 0.9938

EarlyStopping


In [15]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [16]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [17]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=39_auc_mean=0.9220583291148317.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [18]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [19]:
y_probas = np.mean(result, axis=-1)

In [20]:
y_probas[:10]

array([[9.9273829e-04, 2.2985693e-02, 9.7583580e-01, 1.8573811e-04],
       [3.9310761e-02, 1.7064143e-02, 9.4097364e-01, 2.6514246e-03],
       [9.1703532e-06, 1.3889124e-03, 5.8242813e-06, 9.9859619e-01],
       [9.9602592e-01, 6.7340222e-04, 7.6660613e-04, 2.5340687e-03],
       [9.4989018e-06, 2.8524425e-02, 9.7144318e-01, 2.2904725e-05],
       [4.9693507e-01, 1.8131445e-01, 1.2418524e-01, 1.9756520e-01],
       [9.9773788e-01, 7.0325274e-04, 6.9397263e-04, 8.6491025e-04],
       [3.9168470e-05, 5.9642859e-02, 9.4551471e-04, 9.3937242e-01],
       [3.3926888e-05, 4.5833280e-03, 8.5793276e-05, 9.9529696e-01],
       [1.2231877e-04, 1.2349771e-02, 9.8747653e-01, 5.1319435e-05]],
      dtype=float32)

In [21]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.000993,0.022986,0.975836,0.000186
1,Test_1,0.039311,0.017064,0.940974,0.002651
2,Test_2,0.000009,0.001389,0.000006,0.998596
3,Test_3,0.996026,0.000673,0.000767,0.002534
4,Test_4,0.000009,0.028524,0.971443,0.000023


In [22]:
config.submission_file

'01_starter_code.csv'